In [1]:
import numpy as np
import pandas as pd
import os
import multiprocessing as mp
import glob

In [40]:
def blast(fasta, dummy):
    """
    Simple helper to run BLAST. 
    """
    os.system('bash /oak/stanford/groups/horence/george/splash_utils/blast.sh ' + fasta)
    return 


In [41]:
## The user sets a path to a SPLASH input and a working directory.
splash_input = '/oak/stanford/groups/horence/george/blast_internal_parallelization/test/test_data.tsv'
working_dir = '/oak/stanford/groups/horence/george/blast_internal_parallelization/test'

## Create and/or move to the working directory.  
try:
    os.mkdir(working_dir)
    os.chdir(working_dir)
except FileExistsError:
    os.chdir(working_dir)


In [42]:
splash_input = pd.read_csv(splash_input,sep='\t')
most_freq_columns = [i for i in splash_input.columns if 'most_freq_target' in i and 'cnt' not in i]
df = splash_input[['anchor','most_freq_target_1','cnt_most_freq_target_1']].rename(columns={'most_freq_target_1':'target','cnt_most_freq_target_1':'target_count'})
df['target_rank'] = 1
for i in range(2, len(most_freq_columns) + 1): 
    fd = splash_input[['anchor','most_freq_target_'+str(i),'cnt_most_freq_target_'+str(i)]].rename(columns={'most_freq_target_'+str(i):'target','cnt_most_freq_target_'+str(i):'target_count'})
    fd['target_rank'] = i
    df = pd.concat([df,fd])
df = df[df['target'] != '-'].reset_index(drop=True)

In [32]:
workers = int(os.environ['SLURM_JOB_CPUS_PER_NODE']) 

## Split the compactors into chunks of number == workers. 
cs = df['anchor'] + df['target']
cs_inds = np.array_split(cs.index,workers)

## Write FASTAs.
for i in range(len(cs_inds)):
    fasta = open('11blast_'+str(i)+'.fasta','a')
    sel = cs.iloc[cs_inds[i]]
    for ind in sel.index:
        fasta.write('>'+str(i)+'_'+str(ind)+'\n')
        fasta.write(cs[ind]+'\n')
    fasta.close()

In [ ]:
if __name__ == "__main__":
    with mp.Pool(workers) as p:
        outs = p.starmap(blast, [(i,0) for i in glob.glob('11*fasta')])
        


The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:

The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13

  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:
  1) python/3.6.1 => python/2.7.13


The following have been reloaded with a version change:

The followin

In [ ]:
## As with Pfam, load the FASTA input and BLAST output tables. 
## Except cases where the BLAST output is empty. 
outs = glob.glob('*BLAST_out.txt')
ins = glob.glob('*.fasta') 
fasta = pd.read_csv(ins[0],header=None,sep='\t',engine='python')
for i in range(1,len(ins)):
    fasta1 = pd.read_csv(ins[i],header=None,sep='\t',engine='python')
    fasta = pd.concat([fasta,fasta1])
blast = pd.read_csv(outs[0],header=None,sep='\t',engine='python')
for i in range(1,len(outs)):
    try:
        blast1 = pd.read_csv(outs[i],header=None,sep='\t',engine='python')
        blast = pd.concat([blast,blast1])
    except pd.errors.EmptyDataError:
        pass
    
## Define the BLAST output columns concordantly with the specified 'fmt' in the BLAST command.  
blast.columns = 'qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore sseqid sgi sacc slen staxids'.split(' ')
headers = [i[1:] for i in fasta[0] if i[0] == '>']
seqs = [i for i in fasta[0] if i[0] != '>']
fast = pd.DataFrame({'qseqid':headers,'sequence':seqs})
blast['qseqid'] = blast['qseqid'].astype(str)

## Select BLAST hits having e-values < 0.05 and merge hits left onto a table generated from the input FASTA. 
blast = blast[blast['evalue']<0.05]
blast = fast.merge(blast,how='left')


In [ ]:
df = df.merge(blast,how='left')
df.to_csv('SPLASH_output_BLAST_merged.tsv',sep='\t')